# 範例資料

In [1]:
import tensorflow as tf
import numpy as np
import scipy.sparse as sp

from math_graph import get_adj
from math_graph import get_sp_DAD

num_nodes = 5
edges = [(0, 1), (1, 2), (2, 3), (2, 4)]
# 產生 adj
adj = get_adj(edges, num_nodes)
print(f"adj:\n{adj.toarray()}\n")

# 產生 DADs
DAD = get_sp_DAD(adj).toarray()
DAD = tf.convert_to_tensor(DAD, dtype='float32')
DADs = tf.expand_dims(DAD, 0)
print(f"DADs: shape=(num_K, num_nodes, num_nodes)={DADs.shape} type={type(DADs)}\n{DADs.numpy()}\n")

batch_size = 2
num_Fin = 3
X = np.arange(batch_size*num_nodes*num_Fin).reshape([batch_size, num_nodes, num_Fin])
X = tf.convert_to_tensor(X, dtype='float32')
print(f"X: shape=(batch_size, num_nodes, num_Fin)={X.shape} type={type(X)}\n{X.numpy()}\n")

num_Fout = 4
num_K = DADs.shape[0]
W = np.arange(num_Fin*num_Fout).reshape([num_K, num_Fin, num_Fout])
W = tf.convert_to_tensor(W, dtype='float32')
print(f"W: shape=(num_K, num_Fin, num_Fout)={W.shape} type={type(W)}\n{W.numpy()}\n")


b = np.arange(num_Fout).reshape([num_Fout])
b = tf.convert_to_tensor(b, dtype='float32')
print(f"b: shape=(num_Fout)={b.shape} type={type(b)}\n{b.numpy()}")

adj:
[[0. 1. 0. 0. 0.]
 [1. 0. 1. 0. 0.]
 [0. 1. 0. 1. 1.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]]

DADs: shape=(num_K, num_nodes, num_nodes)=(1, 5, 5) type=<class 'tensorflow.python.framework.ops.EagerTensor'>
[[[0.5        0.4082483  0.         0.         0.        ]
  [0.4082483  0.33333334 0.28867513 0.         0.        ]
  [0.         0.28867513 0.25       0.35355338 0.35355338]
  [0.         0.         0.35355338 0.5        0.        ]
  [0.         0.         0.35355338 0.         0.5       ]]]

X: shape=(batch_size, num_nodes, num_Fin)=(2, 5, 3) type=<class 'tensorflow.python.framework.ops.EagerTensor'>
[[[ 0.  1.  2.]
  [ 3.  4.  5.]
  [ 6.  7.  8.]
  [ 9. 10. 11.]
  [12. 13. 14.]]

 [[15. 16. 17.]
  [18. 19. 20.]
  [21. 22. 23.]
  [24. 25. 26.]
  [27. 28. 29.]]]

W: shape=(num_K, num_Fin, num_Fout)=(1, 3, 4) type=<class 'tensorflow.python.framework.ops.EagerTensor'>
[[[ 0.  1.  2.  3.]
  [ 4.  5.  6.  7.]
  [ 8.  9. 10. 11.]]]

b: shape=(num_Fout)=(4,) type=<class 'tensorflow.p

# 張量運算說明

In [2]:
# DADs @ X
print(f"shape: DADs=(num_K, num_nodes, #num_nodes) X=(batch_size, #num_nodes', num_Fin)")
print(f"shape: DADs={DADs.shape} X={X.shape}\n")
DADsX = tf.tensordot(X, DADs, axes=([1], [-1]))
print(f"shape: DADs@X=DADsX=(batch_size, num_Fin, num_K, num_nodes)={DADsX.shape}\n{DADsX.numpy()}\n\n")

# DADs@X @ W
print(f"shape: DADsX=(batch_size, #num_Fin, #num_K, num_nodes) W=(#num_K, #num_Fin, num_Fout)")
print(f"shape: DADsX={DADsX.shape} W={W.shape}\n")
DADsXW = tf.tensordot(DADsX, W, axes=([1, 2], [1, 0]))
print(f"shape: DADsX@W=DADsXW=(batch_size, num_nodes, num_Fout)={DADsXW.shape}\n{DADsXW.numpy()}\n\n")

# DADs@X@W + b
output = tf.add(DADsXW, b)
print(f"shape: DADsXW+b={output.shape}\n{output.numpy()}")

shape: DADs=(num_K, num_nodes, #num_nodes) X=(batch_size, #num_nodes', num_Fin)
shape: DADs=(1, 5, 5) X=(2, 5, 3)

shape: DADs@X=DADsX=(batch_size, num_Fin, num_K, num_nodes)=(2, 3, 1, 5)
[[[[ 1.2247449  2.732051   9.790647   6.6213202  8.121321 ]]

  [[ 2.1329932  3.7623076 11.036428   7.4748735  8.974874 ]]

  [[ 3.0412416  4.7925644 12.28221    8.328427   9.828427 ]]]


 [[[14.84847   18.185902  28.477375  19.424622  20.924622 ]]

  [[15.756718  19.21616   29.723156  20.278175  21.778175 ]]

  [[16.664967  20.246416  30.968937  21.13173   22.63173  ]]]]


shape: DADsX=(batch_size, #num_Fin, #num_K, num_nodes) W=(#num_K, #num_Fin, num_Fout)
shape: DADsX=(2, 3, 1, 5) W=(1, 3, 4)

shape: DADsX@W=DADsXW=(batch_size, num_nodes, num_Fout)=(2, 5, 4)
[[[ 32.861908  39.260887  45.659866  52.058846]
  [ 53.389748  64.67667   75.96359   87.25052 ]
  [142.4034   175.5127   208.62196  241.73125 ]
  [ 96.52692  118.95154  141.37616  163.80078 ]
  [114.52692  141.45154  168.37616  195.30078 ]]

 [

# GraphConvs

In [3]:
class BaseDense(tf.keras.layers.Dense):
    def add_weight_kernel(self, shape, name='kernel', trainable=True):
        return self.add_weight(name=name,
                               shape=shape,
                               trainable=trainable,
                               initializer=self.kernel_initializer,
                               regularizer=self.kernel_regularizer,
                               constraint=self.kernel_constraint,
                               dtype=self.dtype)

    def add_weight_bias(self, shape, name='bias', trainable=True):
        return self.add_weight(name=name,
                               shape=shape,
                               trainable=trainable,
                               initializer=self.bias_initializer,
                               regularizer=self.bias_regularizer,
                               constraint=self.bias_constraint,
                               dtype=self.dtype)

class GraphConvs(BaseDense):
    def __init__(self, units, DADs, name=None, **kwargs):
        super(GraphConvs, self).__init__(units=units, name=name, **kwargs)
        self.DADs = tf.convert_to_tensor(DADs)
        self.num_K = len(DADs)  # K

    def build(self, input_shape):
        self.num_Fin = input_shape[-1]  # Fin
        self.num_Fout = self.units  # Fout
        self.kernel = self.add_weight_kernel([self.num_K, self.num_Fin, self.num_Fout])
        self.bias = self.add_weight_bias([self.num_Fout,]) if self.use_bias else None
        self.built = True

    def call(self, inputs):
        X = tf.tensordot(inputs, self.DADs, axes=([1], [-1]))
        X = tf.tensordot(X, self.kernel, axes=([1, 2], [1, 0]))
        if self.use_bias:
            X = tf.add(X, self.bias)
        outputs = self.activation(X)
        return outputs

In [4]:
from layer_gconv import GraphConvs

## 產生 adj  #使用 無向邊 建構
# adj = get_adj(edges, num_nodes)

## 產生 DADs
# DAD = get_sp_DAD(adj).toarray()
# DAD = tf.convert_to_tensor(DAD, dtype='float32')
# DADs = tf.expand_dims(DAD, 0)

GraphConvs(units=num_Fout, DADs=DADs, use_bias=True)(X)

<tf.Tensor: shape=(2, 5, 4), dtype=float32, numpy=
array([[[ 4.8153954 ,  3.1498032 ,  2.5675948 , -0.09882069],
        [ 8.212198  ,  4.9054947 ,  4.0464797 , -0.4879012 ],
        [23.215256  , 12.369419  , 10.371268  , -2.4055085 ],
        [15.728307  ,  8.388839  ,  7.032632  , -1.6238561 ],
        [18.800213  ,  9.877428  ,  8.299372  , -2.0436625 ]],

       [[32.715935  , 16.669893  , 14.072742  , -3.9117055 ],
        [39.860725  , 20.241793  , 17.097158  , -4.8129864 ],
        [61.484512  , 30.913998  , 26.152088  , -7.6353817 ],
        [41.948673  , 21.094748  , 17.844938  , -5.2071285 ],
        [45.020576  , 22.583336  , 19.11168   , -5.626935  ]]],
      dtype=float32)>